# **LangGraph Research Assistant Mockup**

Este ejercicio ejemplifica la arquitectura de un agente en LangGraph, sin integrar LLMs ni herramientas externas. No pretende construir un agente operativo, sino descomponer la orquestación de flujos complejos mediante grafos. A partir de un mockup funcional, se abordan el estado compartido, los nodos, el enrutamiento condicional y el ensamblaje del grafo. Este enfoque permite comprender la estructura y la mecánica de paso de mensajes que habilitan agentes autónomos, separando la lógica de orquestación de la complejidad de las APIs reales.

## **Ejemplo de un caso de uso: Asistente de investigación científica**
Imaginemos que necesitamos construir un asistente de investigación académica que ayude a científicos a encontrar artículos de investigación relevantes sobre un tema particular. Este caso describe conceptualmente la construcción de un asistente académico capaz de transformar preguntas en lenguaje natural en hallazgos verificables. La solución prioriza trazabilidad y control, cada decisión del sistema queda guardada en un estado persistente, y los pasos críticos se someten a validación humana antes de la entrega final.

## **1. Instalación de dependencias**

Primero, instalamos las librerías necesarias para que el grafo de LangGraph y sus utilidades funcionen correctamente.

In [18]:
!pip install langgraph>=1.0.0 Pillow langsmith

## **2. Definición de clases y funciones de utilidad**

A continuación, definimos todo el código de los módulos auxiliares (`state.py`, `utils.py`, `graph_utils.py`) en celdas separadas para mantener el orden.

### **Definición de la Memoria o el estado del agente**
Antes de que cualquier acción ocurra en el grafo en LangGraph, se necesita definir la memoria o el estado que compartirá todo el agente. Se utiliza `TypedDict` de Python para crear una estructura llamada `ResearchState`, que actúa como el contenedor central de datos.

`ResearchState`: comienza conteniendo solo el tema de investigación. A medida que pasa por los diferentes pasos, cada nodo lee esta estructura, realiza su tarea (como generar queries o evaluar si los resultados son suficientes) y añade o actualiza la información antes de pasarla al siguiente nodo.

In [19]:
from typing import List, TypedDict

class ResearchState(TypedDict):
    """
    Representa el estado actual del proceso de investigación.
    """
    topic: str
    queries: List[str]
    raw_results: List[str]
    iteration_count: int
    is_sufficient: bool
    report_draft: str
    human_feedback: str

La función `simular_carga` está diseñada para simular un retraso o una operación de carga en la consola, mostrando una barra de progreso animada.

In [20]:
import sys
import time

def simular_carga(segundos: int = 3, mensaje: str = "Procesando"):
    """
    Simula un retraso con una barra de carga animada en la consola.
    """
    ancho_barra = 30
    intervalo = segundos / ancho_barra

    sys.stdout.write(f"\n{mensaje} [")
    sys.stdout.flush()

    for _ in range(ancho_barra):
        time.sleep(intervalo)
        sys.stdout.write("=")
        sys.stdout.flush()

    sys.stdout.write("] Listo!\n")
    sys.stdout.flush()
    time.sleep(0.5)

La función `save_graph_image` toma una instancia de `StateGraph`y un nombre de archivo como entrada. Su propósito es generar una representación visual de ese grafo y guardarla como un archivo de imagen PNG. Internamente, utiliza el método `draw_mermaid_png(`) del grafo para obtener los bytes de la imagen. Esto es útil para depurar, documentar o simplemente entender la estructura de un flujo de trabajo de LangGraph.

In [21]:
from PIL import Image
import io
from langgraph.graph import StateGraph

def save_graph_image(graph: StateGraph, filename: str = "research_assistant_mockup.png"):
    """
    Genera y guarda una imagen del grafo de LangGraph en formato PNG.
    """
    try:
        image_bytes = graph.get_graph().draw_mermaid_png()
        image = Image.open(io.BytesIO(image_bytes))
        image.save(filename)
        print(f"\nImagen del grafo guardada como '{filename}'")
    except Exception as e:
        print(f"(No se pudo generar la imagen del grafo por falta de dependencias opcionales o error: {e})")

## 3. Lógica Principal del Grafo de Investigación

Esta es la celda principal que contiene la lógica del `main.py` original. Aquí se definen los nodos, las aristas y la lógica condicional del grafo.

### **Nodo generar consulta**
El primer paso operativo del flujo en el grafo es el nodo `node_generate_queries`, que actúa como el punto de partida del agente. Su función es tomar el tema general de investigación almacenado en el estado y traducirlo en acciones concretas: una lista de consultas de búsqueda.

En este bloque, simulamos el comportamiento de una IA compleja que '***piensa***' (representado por la barra de carga) para descomponer el tema principal en sub-búsquedas estratégicas, como buscar principios básicos, avances recientes o metodologías relacionadas. Al finalizar su tarea, este nodo actualiza nuestro estado compartido con estas nuevas consultas y reinicia los contadores de seguimiento, dejando todo listo para que el siguiente nodo ejecute la búsqueda.

In [22]:
import operator
from typing import Annotated, List, Literal, Union
from langgraph.graph import END, START, StateGraph

# --- Nodos del Grafo (Mockups Funcionales) ---
def node_generate_queries(state: ResearchState) -> ResearchState:
    simular_carga(3, "Iniciando motor de razonamiento")
    print(f"\n\n===== [node_generate_queries] =====\n")
    print(f"   - Tema de Investigación: '{state['topic']}'")
    new_queries = [f"principios {state['topic']}", f"avances recientes {state['topic']}", f"{state['topic']} metodología"]
    print(f"   - Consultas Generadas: {new_queries}")
    return {
        "queries": new_queries,
        "iteration_count": 0,
        "human_feedback": "pending"
    }

### **Nodos de búsqueda**
Una vez que tenemos la lista de consultas de búsqueda, el agente entra en un ciclo crítico de búsqueda y evaluación, representado por los nodos `node_search_api` y` node_evaluate_results`. Primero, `node_search_api` toma las consultas del estado y simula una conexión a bases de datos académicas externas, devolviendo una lista preliminar de documentos.

In [23]:
def node_search_api(state: ResearchState) -> ResearchState:
    simular_carga(3, "Consultando bases de datos externas")
    print(f"\n\n===== [node_search_api] =====\n")
    print(f"   - Ejecutando Consultas: {state['queries']}")
    if state["iteration_count"] == 0:
        results = ["Paper A (muy antiguo)", "Paper B (poco relevante)"]
    else:
        results = ["Paper A", "Paper B", "Paper C (relevante, 2024)", "Paper D (survey seminal)"]
    print(f"   - Resultados Encontrados: {len(results)} documentos.")
    return {"raw_results": results}

### **Nodo de evaluación de resultados**
Después, `node_evaluate_results` actúa como un filtro de calidad, analiza si la cantidad y relevancia de estos resultados cumplen con nuestros criterios mínimos definidos (en este caso, simulado por un umbral de tres documentos). Este nodo no solo determina si podemos avanzar a la fase de escritura, sino que también incrementa el contador de iteraciones, un mecanismo clave para evitar que nuestro asistente se quede atrapado en un bucle de búsqueda infinito si no logra encontrar suficiente información.

In [24]:
def node_evaluate_results(state: ResearchState) -> ResearchState:
    simular_carga(2, "Analizando relevancia")
    print(f"\n\n===== [node_evaluate_results] =====\n")
    is_sufficient = len(state["raw_results"]) >= 3
    if not is_sufficient:
        print("   - Evaluación: INSUFICIENTE. Se requiere refinamiento.")
    else:
        print("   - Evaluación: SUFICIENTE. Procediendo a síntesis.")
    return {
        "is_sufficient": is_sufficient,
        "iteration_count": state["iteration_count"] + 1
    }

### **Nodo de refinar busqueda**
Si la evaluación inicial resulta insuficiente, el flujo se desvía hacia `node_refine_search`. Este nodo actúa como un estratega que toma las consultas originales y las modifica (por ejemplo, añadiendo términos como review para buscar artículos de revisión más amplios), preparándolas para una nueva ronda de búsqueda.

In [25]:
def node_refine_search(state: ResearchState) -> ResearchState:
    simular_carga(2, "Ajustando parámetros de búsqueda")
    print(f"\n\n===== [node_refine_search] =====\n")
    new_queries = [q + " review" for q in state["queries"]]
    print(f"   - Nuevas Consultas Generadas: {len(new_queries)}")
    return {"queries": new_queries}

### **Nodo de sintetizar reporte**
Una vez que los resultados son satisfactorios, el control pasa a `node_synthesize_report`. Este nodo toma toda la información recopilada y redacta un borrador preliminar, estructurando los hallazgos clave en un texto coherente.


In [26]:
def node_synthesize_report(state: ResearchState) -> ResearchState:
    simular_carga(4, "Redactando informe final")
    print(f"\n\n===== [node_synthesize_report] =====\n")
    draft = f"INFORME SOBRE {state['topic'].upper()}\nBasado en {len(state['raw_results'])} papers clave, el campo muestra..."
    print("   - Borrador de Informe Generado.")
    return {"report_draft": draft}

### **Nodo de aprobación del humano**
Finalmente, el proceso llega a `node_human_approval`, un que introduce al **human-in-the-loop**. Aquí, el sistema pausa (simbólicamente en este mockup) para presentar el borrador al usuario, esperando su veredicto final (aprobación o rechazo) antes de dar por terminada la tarea.


In [27]:
def node_human_approval(state: ResearchState) -> ResearchState:
    simular_carga(1, "Preparando interfaz de revisión")
    print(f"\n\n===== [node_human_approval] =====\n")
    print(f"   - Mostrando borrador al usuario para revisión:\n\n{state['report_draft']}\n")
    simular_carga(2, "Esperando input del usuario")
    print("\n   >>> [Simulación] Usuario revisando... APROBADO.\n")
    return {"human_feedback": "approve"}

### **Autonomía del agente**
Para implementar autonomía al agente, se incorporan enrutadores o nodos de decisión lógica, que actúan como mecanismos de control que regulan el flujo del proceso en función de la información disponible en el estado. El componente `route_evaluation` constituye el primer punto de control, analiza la bandera de suficiencia y determina si el sistema puede avanzar hacia la fase de redacción o si, por el contrario, debe retornar a etapas previas para refinar los resultados.

In [28]:
# --- Lógica Condicional (Routers) ---
def route_evaluation(state: ResearchState) -> Literal["synthesize_report", "refine_search"]:
    if state["is_sufficient"]:
        return "synthesize_report"
    return "refine_search"

### **Decisión final**
De manera complementaria, el componente `route_human_feedback` gestiona la decisión final a partir de la intervención humana; cuando el revisor aprueba el borrador, el flujo se dirige al nodo `END`, concluyendo el proceso, y en caso contrario se ordena un reinicio del ciclo de investigación. Estas bifurcaciones condicionales convierten una secuencia previamente lineal en un sistema dinámico y adaptable, capaz de ajustar su trayectoria en función del desempeño observado.

In [29]:
def route_human_feedback(state: ResearchState) -> Literal[END, "generate_queries"]:
    if state["human_feedback"] == "approve":
        print("\n\n===== PROCESO FINALIZADO CON ÉXITO =====\n")
        print("   - Informe entregado satisfactoriamente.\n")
        return END
    print("\n\n===== FEEDBACK NEGATIVO =====\n")
    print("   - Reiniciando proceso de investigación.\n")
    return "generate_queries"

### **Construcción del grafo**
En la fase de construcción del grafo se integran los componentes operativos dentro de una arquitectura ejecutable. Se inicializa el `StateGraph` que es el constructor utilizado para definir el flujo de trabajo del agente y al pasar ResearchState como argumento, se define que todos los nodos en este grafo deben recibir y devolver datos que se ajusten a esa estructura específica (`TypedDict`).

A continuación, se traza la topología del flujo de datos, se emplean aristas (`add_edge`) para establecer secuencias determinísticas y aristas condicionales (`add_conditional_edges`) en los puntos de control donde los enrutadores deben seleccionar dinámicamente el siguiente estado (ejemplo, evaluar calidad o esperar aprobación).

La invocación final a `compile()` construye una máquina de estados lista para la ejecución de tareas.

In [30]:
# --- Construcción del Grafo ---
builder = StateGraph(ResearchState)
builder.add_node("generate_queries", node_generate_queries)
builder.add_node("search_api", node_search_api)
builder.add_node("evaluate_results", node_evaluate_results)
builder.add_node("refine_search", node_refine_search)
builder.add_node("synthesize_report", node_synthesize_report)
builder.add_node("human_approval", node_human_approval)
builder.add_edge(START, "generate_queries")
builder.add_edge("generate_queries", "search_api")
builder.add_edge("search_api", "evaluate_results")
builder.add_conditional_edges("evaluate_results", route_evaluation)
builder.add_edge("refine_search", "search_api")
builder.add_edge("synthesize_report", "human_approval")
builder.add_conditional_edges("human_approval", route_human_feedback)
graph = builder.compile()

### **Ejecución del grafo**
Finalmente se instancia el ResearchState inicial, especificando el tema de investigación a explorar (ejemplo, el impacto de la IA en la educación) y manteniendo vacíos los campos restantes, a modo de estado inicial.

A continuación, una llamada a graph.invoke(initial_state) inicia la ejecución. La función toma el estado inicial e inyecta la información en el nodo de arranque (START), ejecutando el grafo, en el que cada nodo cumple con su función y transfiere el control al siguiente hasta finalizar el proceso.

In [31]:
# --- Ejecución y Visualización ---
save_graph_image(graph, "research_assistant_mockup.png")
initial_state = ResearchState(
    topic = "Impacto de la IA generativa en la educación superior",
    queries = [], raw_results = [], iteration_count = 0,
    is_sufficient = False, report_draft="", human_feedback = "pending"
)
print("\n\n************************************************")
print("***** Iniciando Asistente de Investigación *****")
print("************************************************\n")
final_state = graph.invoke(
    initial_state,
    config={"tags": ["research_workflow", "generative_ai_education"]}
)


Imagen del grafo guardada como 'research_assistant_mockup.png'


************************************************
***** Iniciando Asistente de Investigación *****
************************************************


Iniciando motor de razonamiento [==============================] Listo!


===== [node_generate_queries] =====

   - Tema de Investigación: 'Impacto de la IA generativa en la educación superior'
   - Consultas Generadas: ['principios Impacto de la IA generativa en la educación superior', 'avances recientes Impacto de la IA generativa en la educación superior', 'Impacto de la IA generativa en la educación superior metodología']

Consultando bases de datos externas [==============================] Listo!


===== [node_search_api] =====

   - Ejecutando Consultas: ['principios Impacto de la IA generativa en la educación superior', 'avances recientes Impacto de la IA generativa en la educación superior', 'Impacto de la IA generativa en la educación superior metodología']
   - Res